In [1]:
import pandas as pd

df = pd.read_csv('public_result_label.csv')

# 질문 리스트
questions = df['question'].tolist()
answers = df['answer'].tolist()

df.head()

,question,answer,category02,label
0,여기 서명란에 펜이나 연필로 사인해도 되는 거 아니에요?,확인 결과 인감 외 사인도 동일 서명으로 처리됩니다.,민원 신청서 용어 질문,0
1,이런 전기 요금 할인처럼 비슷한 서비스 같은 것이 또 있나요?,네 소득기준과 세대원 특정 기준에 모두 충족하는 세대에게 에너지 바우처를 지원해 줍니다.,관내 복지 지원금 정보 질문,1
2,여기 소포 보낼 때 한 번에 결제하고 그런 서비스는 지원 안 하나요?,죄송합니다. 선불로 처리될 수밖에 없는 점 양해 부탁드립니다.,관내 복지 지원금 정보 질문,1
3,이런데 로봇이 있어? 신기하네 사업 지원금 관련 담당자는 어디 계실까요?,담당자분 성함 알려주시면 확인 후 안내 도와드리겠습니다.,입장 및 민원서비스 이용 안내,2
4,서류 가지고 왔어요. 여기에 제출하면 되죠?,성적장학금의 경우 재학 증명서와 성적 증명서가 필요합니다.,준비서류 확인,3


In [2]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
# 모델 구조를 정의한 코드
model = AutoModelForSequenceClassification.from_pretrained("beomi/KcBERT-base", num_labels=6)

# 모델 가중치를 CPU로 로드
model.load_state_dict(torch.load('model/public_model_weights.pth', map_location=torch.device('cpu')))
tokenizer = AutoTokenizer.from_pretrained("beomi/KcBERT-base")

c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/KcBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Temp\ipykernel_12132\3226958015.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `

In [4]:
import torch
import torch.nn.functional as F
import numpy as np

# numpy의 _reconstruct 함수를 허용 목록에 추가
torch.serialization.add_safe_globals([np.core.multiarray._reconstruct])

# 데이터프레임과 디바이스 설정
device = torch.device('cpu')

# 신뢰할 수 있는 데이터 파일을 로드하고 GPU로 이동
question_embeddings = torch.load('embeddings/public_train_question_embeddings.pth')
question_embeddings = [torch.tensor(embedding).to(device) for embedding in question_embeddings]  # 모든 임베딩을 GPU로 이동

# 평가 모드로 설정
model.eval()


def get_embedding(input_question, tokenizer, model, device):
    # 입력 문장을 토크나이즈하고 GPU/CPU로 이동
    inputs = tokenizer(input_question, return_tensors="pt", padding=True, truncation=True).to(device)
    
    with torch.no_grad():
        # hidden states를 포함하도록 설정
        outputs = model(**inputs, output_hidden_states=True)
        
        # 마지막 hidden state에서 [CLS] 토큰의 임베딩을 가져옴
        cls_embedding = outputs.hidden_states[-1][:, 0, :]  # [CLS] 토큰의 임베딩
    
    return cls_embedding.squeeze().to(device)  # 텐서를 GPU로 반환

# 코사인 유사도를 계산하여 가장 유사한 답변을 찾는 함수
def find_most_similar_answer_cosine(input_question, question_embeddings, answers, tokenizer, model, device):
    # 입력 질문 임베딩 생성
    input_embedding = get_embedding(input_question, tokenizer, model, device)

    max_similarity = -1
    best_answer = None
    
    # 각 질문 임베딩과 유사도 비교
    for i, question_embedding in enumerate(question_embeddings):
        # 코사인 유사도 계산
        similarity = F.cosine_similarity(input_embedding.unsqueeze(0), question_embedding.unsqueeze(0)).item()
        
        if similarity > max_similarity:
            max_similarity = similarity
            best_answer = answers[i]

    return best_answer, max_similarity

# 챗봇 응답 함수
def chatbot_response(input_question, tokenizer, model, question_embeddings, answers, df, device):
    # 1차 필터링: 분류 모델로 레이블 예측
    inputs = tokenizer(input_question, return_tensors="pt", padding=True, truncation=True).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=-1).item()

    # 2차 필터링: 같은 카테고리 내에서 코사인 유사도 계산
    # 같은 레이블의 질문들과 임베딩 필터링
    filtered_df = df[df['label'] == predicted_label]
    filtered_indices = filtered_df.index.tolist()

    # 필터링된 질문에 해당하는 미리 계산된 임베딩과 답변 가져오기
    filtered_question_embeddings = [question_embeddings[i] for i in filtered_indices]
    filtered_answers = [answers[i] for i in filtered_indices]

    # 코사인 유사도를 통해 가장 유사한 답변 찾기
    best_answer, cosine_similarity = find_most_similar_answer_cosine(input_question, filtered_question_embeddings, filtered_answers, tokenizer, model, device)
    
    return best_answer, cosine_similarity, predicted_label


c:\Temp\ipykernel_12132\594036569.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  question_embeddings = torch.load('embeddings/public_train_question_embeddings.pth')


In [5]:
# 예시 질문
input_question = "서류 제출은 어디에 해요?"

# 챗봇 응답 호출
best_answer, cosine_similarity, predicted_label = chatbot_response(
    input_question, tokenizer, model, question_embeddings, answers, df, device
)

# 결과 출력

print("예측된 레이블:", predicted_label)
print("최고 유사도 답변:", best_answer)
print("코사인 유사도:", cosine_similarity)

예측된 레이블: 4
최고 유사도 답변: 말씀하신 질문에 대한 답변으로, 상담 문의는 능력 개발 운영부입니다.
코사인 유사도: 0.8322391510009766
